# Test OpenAI Agents SDK
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

17:11:40 | NewsletterAgent.newsletter_agent | INFO | Test info message
17:11:40 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
17:11:40 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))
try:
    # set up state
    session_id = 'test_newsletter_20250928144400108448'
    step_name = 'step_05_cluster_by_topic'
#     del session_id
except Exception as e:
    print(e)

do_download=False
process_since=None
# process_since='2025-09-24 18:00:00'

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=True
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=True
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

17:11:49 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Using provided state with 313 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20250928144400108448
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20250928144400108448


In [6]:
agent.state.get_status()


{'headlines': {'total': 313},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_06_rate_articles',
  'workflow_complete': False,
  'workflow_status': 'started',
  'workflow_status_message': '',
  'progress_percentage': 55.55555555555556,
  'max_edits': 2,
  'concurrency': 16},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [7]:
state.get_current_step()


'step_06_rate_articles'

In [11]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


14:48:33 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Starting check_workflow_status
14:48:33 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Completed check_workflow_status


⏱️  Total execution time: 7.02s
📊 Final result:
Current workflow status:
- Progress: 0.0% (0/9 complete)
- 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch URLs

Step details:
- Step 1: Fetch Urls — not_started
- Step 2: Filter Urls — not_started
- Step 3: Download Articles — not_started
- Step 4: Extract Summaries — not_started
- Step 5: Cluster By Topic — not_started
- Step 6: Rate Articles — not_started
- Step 7: Select Sections — not_started
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

Would you like me to start Step 1 (gather headlines and URLs) or resume/execute a specific step?


In [12]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


14:48:41 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Starting check_workflow_status
14:48:41 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Completed check_workflow_status
14:48:43 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Starting Step 1: Gather URLs
2025-09-28 14:48:43,066 - fetcher_5472532496 - INFO - [fetcher_init] Loading sources from sources.yaml
2025-09-28 14:48:43,079 - fetcher_5472532496 - INFO - [fetcher_init] Loaded 17 sources: 7 RSS, 9 HTML, 1 API
2025-09-28 14:48:43,079 - fetcher_5472532496 - DEBUG - [fetcher_sources] Source 'Ars Technica': type=RSS, url=https://arstechnica.com/ai/
2025-09-28 14:48:43,079 - fetcher_5472532496 - DEBUG - [fetcher_sources] Source 'Bloomberg': type=RSS, url=https://www.bloomberg.com/ai
2025-09-28 14:48:43,080 - fetcher_5472532496 - DEBUG - [fetcher_sources] Source 'Business Insider': type=html, url=https://www.businessinsider.com/tech
2025-09-28 14:48:43,080 - fetcher_5472532496 - DEBU

▶ Starting Step 1: step_01_fetch_urls


2025-09-28 14:48:43,322 - fetcher_5472532496 - INFO - [fetch_html] Parsed HTML file: download/sources/Feedly_AI.html
2025-09-28 14:48:43,322 - fetcher_5472532496 - INFO - [fetch_html] HTML fetch successful for Feedly AI: 113 articles
2025-09-28 14:48:43,322 - fetcher_5472532496 - INFO - [fetch_rss] Fetching RSS from Hacker News: https://news.ycombinator.com/rss
2025-09-28 14:48:43,323 - fetcher_5472532496 - INFO - [fetch_rss] Fetching RSS from HackerNoon: https://hackernoon.com/tagged/ai/feed
2025-09-28 14:48:43,323 - fetcher_5472532496 - INFO - [fetch_rss] Fetching RSS from New York Times: https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml
2025-09-28 14:48:43,323 - fetcher_5472532496 - INFO - [fetch_html] Using existing HTML file from Reddit: https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+

,source,url
0,Ars Technica,20
1,Bloomberg,27
2,Business Insider,15
3,FT,47
4,Feedly AI,109
5,Hacker News,29
6,HackerNoon,50
7,New York Times,23
8,NewsAPI,49
9,Reddit,48


,source,title,url,published,rss_summary,id
0,Ars Technica,Why LA Comic Con thought making an AI-powered ...,https://arstechnica.com/ai/2025/09/why-la-comi...,"Sat, 27 Sep 2025 11:00:07 +0000",“I suppose if we do it and thousands of fans… ...,0
1,Ars Technica,Can AI detect hedgehogs from space? Maybe if y...,https://arstechnica.com/ai/2025/09/can-ai-dete...,"Fri, 26 Sep 2025 22:22:13 +0000",Cambridge researchers use satellite-based bram...,1
2,Ars Technica,YouTube Music is testing AI hosts that will in...,https://arstechnica.com/google/2025/09/youtube...,"Fri, 26 Sep 2025 21:05:30 +0000",YouTube Labs will be a place to preview all th...,2
3,Ars Technica,ChatGPT Pulse delivers morning updates based o...,https://arstechnica.com/ai/2025/09/chatgpt-pul...,"Thu, 25 Sep 2025 20:30:52 +0000",New mobile chatbot feature analyzes conversati...,3
4,Ars Technica,Experts urge caution about using ChatGPT to pi...,https://arstechnica.com/information-technology...,"Thu, 25 Sep 2025 18:10:50 +0000",AI-selected portfolios might perform well in a...,4
...,...,...,...,...,...,...
581,NewsAPI,langstruct added to PyPI,https://pypi.org/project/langstruct/,2025-09-27T18:05:09Z,NaN,581
582,NewsAPI,When teachers learn too,https://www.thestar.com.my/news/education/2025...,2025-09-27T16:00:00Z,NaN,582
583,NewsAPI,M’sian education system a work in progress,https://www.thestar.com.my/news/education/2025...,2025-09-27T16:00:00Z,NaN,583
584,NewsAPI,When PhDs are not enough,https://www.thestar.com.my/news/education/2025...,2025-09-27T16:00:00Z,NaN,584


14:48:44 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Completed Step 1: Gathered 677 articles
14:48:45 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Starting check_workflow_status
14:48:45 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Completed check_workflow_status


⏱️  Total execution time: 10.52s
📊 Final result:
Step 1 completed successfully.

Summary:
- Fetched headlines and URLs from RSS sources.
- Articles gathered: 677 found across 17 sources; 586 articles stored in persistent state.
- Workflow progress: 1/9 steps complete (11.1%).
- Next step: Step 2 — Filter URLs (AI-related content)

Would you like me to continue and run Step 2 now?


In [13]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,Why LA Comic Con thought making an AI-powered ...,https://arstechnica.com/ai/2025/09/why-la-comi...,"Sat, 27 Sep 2025 11:00:07 +0000",“I suppose if we do it and thousands of fans… ...,0
1,Ars Technica,Can AI detect hedgehogs from space? Maybe if y...,https://arstechnica.com/ai/2025/09/can-ai-dete...,"Fri, 26 Sep 2025 22:22:13 +0000",Cambridge researchers use satellite-based bram...,1
2,Ars Technica,YouTube Music is testing AI hosts that will in...,https://arstechnica.com/google/2025/09/youtube...,"Fri, 26 Sep 2025 21:05:30 +0000",YouTube Labs will be a place to preview all th...,2
3,Ars Technica,ChatGPT Pulse delivers morning updates based o...,https://arstechnica.com/ai/2025/09/chatgpt-pul...,"Thu, 25 Sep 2025 20:30:52 +0000",New mobile chatbot feature analyzes conversati...,3
4,Ars Technica,Experts urge caution about using ChatGPT to pi...,https://arstechnica.com/information-technology...,"Thu, 25 Sep 2025 18:10:50 +0000",AI-selected portfolios might perform well in a...,4
...,...,...,...,...,...,...
581,NewsAPI,langstruct added to PyPI,https://pypi.org/project/langstruct/,2025-09-27T18:05:09Z,NaN,581
582,NewsAPI,When teachers learn too,https://www.thestar.com.my/news/education/2025...,2025-09-27T16:00:00Z,NaN,582
583,NewsAPI,M’sian education system a work in progress,https://www.thestar.com.my/news/education/2025...,2025-09-27T16:00:00Z,NaN,583
584,NewsAPI,When PhDs are not enough,https://www.thestar.com.my/news/education/2025...,2025-09-27T16:00:00Z,NaN,584


In [14]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
4,Feedly AI,109
12,The Register,50
6,HackerNoon,50
8,NewsAPI,49
9,Reddit,48
3,FT,47
5,Hacker News,29
16,Washington Post,28
1,Bloomberg,27
7,New York Times,23


In [15]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


14:49:09 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Starting Step 2: Filter URLs
14:49:09 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | 🔍 Filtering 586 headlines...
14:49:09 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | 🔍 Filtering 586 for dupes.
14:49:09 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | 🔍 Filtering 586 headlines for AI relevance using LLM...
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=459, user_len=954


▶ Starting Step 2: step_02_filter_urls


14:49:25 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Completed Step 2: 326 AI-related articles


⏱️  Total execution time: 16.70s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Filtered 586 headlines to 326 AI-related articles.


In [16]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

14:49:48 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Starting Step 3: Download Articles
14:49:48 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Starting concurrent scraping of 326 AI-related articles


▶ Starting Step 3: step_03_download_articles


14:49:48 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Launching browser for 326 URLs with 16 concurrent workers
14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Worker 0 fetching 1 of 326 https://www.scmp.com/tech/big-tech/article/3327138/china-nanoseconds-behind-us-chips-says-nvidias-jensen-huang
14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | scrape_url(https://www.scmp.com/tech/big-tech/article/3327138/china-nanoseconds-behind-us-chips-says-nvidias-jensen-huang)
14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | scraping https://www.scmp.com/tech/big-tech/article/3327138/china-nanoseconds-behind-us-chips-says-nvidias-jensen-huang to download/html
14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Downloading https://www.scmp.com/tech/big-tech/article/3327138/china-nanoseconds-behind-us-chips-says-nvidias-jensen-huang
14:49:50 | NewsletterAgent.test_newsletter_2025092814440

14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Worker 8 completed https://www.washingtonpost.com/business/2025/09/02/ai-skills-jobs-tips/ with status: success
14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Worker 8 fetching 14 of 326 https://venturebeat.com/ai/chinese-food-delivery-firm-meituans-open-source-ai-model-longcat-flash
14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | File already exists: download/html/Chinese_food_delivery_app_Meituan_s_open_source_AI_model_LongCat-Flash-Thinking_rivals_GPT-5.html
14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Worker 8 completed https://venturebeat.com/ai/chinese-food-delivery-firm-meituans-open-source-ai-model-longcat-flash with status: success
14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Worker 8 fetching 15 of 326 https://hackernoon.com/gemini-might-be-the-only-actual-foundational-model-out-there?source=rss
14:

14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Skipping ignored domain: www.bloomberg.com
14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Worker 13 completed https://www.bloomberg.com/news/articles/2025-09-26/ai-and-health-care-uk-grapples-with-regulation-while-seeking-investment with status: success
14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Worker 13 fetching 26 of 326 https://www.businessinsider.com/goldman-sachs-marco-argenti-cio-interview-ai-engineers-careers-2025-9
14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | scrape_url(https://www.businessinsider.com/goldman-sachs-marco-argenti-cio-interview-ai-engineers-careers-2025-9)
14:49:50 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | scraping https://www.businessinsider.com/goldman-sachs-marco-argenti-cio-interview-ai-engineers-careers-2025-9 to download/html
14:49:50 | NewsletterAgent.test_newsletter_202509281444

...

15:41:22 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Worker 11 fetching 320 of 326 https://www.wsj.com/tech/ai/world-models-ai-evolution-11275913
15:41:22 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Skipping ignored domain: www.wsj.com
15:41:22 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Worker 11 completed https://www.wsj.com/tech/ai/world-models-ai-evolution-11275913 with status: success
15:41:22 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Worker 11 fetching 321 of 326 https://seekingalpha.com/article/4826240-my-absolute-favorite-picks-for-the-4-trillion-ai-infrastructure-boom
15:41:22 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | scrape_url(https://seekingalpha.com/article/4826240-my-absolute-favorite-picks-for-the-4-trillion-ai-infrastructure-boom)
15:41:22 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | scraping https://seekingalpha.com/article/4826240-my-absolute-favo

15:42:33 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Worker 4 completed https://futurism.com/health-medicine/experts-alarmed-ai-viruses with status: success
15:42:33 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Saving HTML to download/html/Rhombus_Power_Snags__200_Million_US_Air_Force_Contract_for_AI-Powered_Strategic_Decision_Making_Platform.html
15:42:33 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Saving HTML to download/html/Box_s_Explosive_AI_Potential__NYSE_BOX.html
15:42:33 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Worker 9 completed https://theaiinsider.tech/2025/09/27/rhombus-power-snags-200-million-us-air-force-contract-for-ai-powered-strategic-decision-making-platform/ with status: success
15:42:34 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Worker 3 completed https://seekingalpha.com/article/4826458-boxs-explosive-ai-potential with status: success
15:42:34 | NewsletterAgent.t

Starting with 323 rows...
Processing 323 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2

15:43:38 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Completed Step 3: Downloaded 301 articles


Filtering dataframe...
  Pair: 214 vs 124
    214: Techmeme - iRobot co-founder Rodney Brooks details why humanoid robots won't learn human-level dexterity from current methods, how to make them safe for humans, and more (Rodney Brooks)
    124: Feedly AI - iRobot co-founder Rodney Brooks details why humanoid robots won't learn human-level dexterity from current methods, how to make them safe for humans, and more (Rodney Brooks)
  Pair: 218 vs 162
    218: The Register - Alibaba unveils $53B global AI plan – but it will need GPUs to back it up
    162: Feedly AI - Alibaba unveils $53B global AI plan – but it will need GPUs to back it up
  Pair: 215 vs 125
    215: Techmeme - At the UN, the US rejected calls for collaborative efforts around AI governance, even as many leaders endorsed a need for urgent international collaboration (Jared Perlo/NBC News)
    125: Feedly AI - At the UN, the US rejected calls for collaborative efforts around AI governance, even as many leaders endorsed a ne

In [17]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

15:52:52 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Processing 313 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1204, user_len=43
15:52:52 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Using model 'gpt-4.1-mini' for summarization
15:52:52 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Starting summarization for 313 articles


▶ Starting Step 4: step_04_extract_summaries


15:53:01 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Extracting metadata from HTML files for 313 articles


⏱️  Total execution time: 15.78s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 313/313 articles.
💾 Summaries stored in headline DataFrame.


In [19]:
headline_df = state.headline_df
headline_df.loc[headline_df["url"]!= headline_df["final_url"]]

,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags
6,Feedly AI,Harvard's BKC Explores Whether Human Intellige...,https://news.google.com/rss/articles/CBMi7AFBV...,NaN,NaN,166,True,success,https://www.forbes.com/sites/lanceeliot/2025/0...,download/html/Harvard_s_BKC_Explores_Whether_H...,2025-09-28T07:15:00Z,download/text/Harvard_s_BKC_Explores_Whether_H...,17406,forbes.com,Forbes,1.0,- Harvard's Berkman Klein Center hosted a Fall...,Is human intelligence really computational int...,"[intelligence cognition thinking neurons, Anth..."
15,Feedly AI,Brain Surgeon Shows Why AI Can Never Become Human,https://news.google.com/rss/articles/CBMiiAFBV...,NaN,NaN,197,True,success,https://mindmatters.ai/2025/09/brain-surgeon-s...,download/html/Brain_Surgeon_Shows_Why_AI_Can_N...,2025-09-27T21:34:26Z,download/text/Brain_Surgeon_Shows_Why_AI_Can_N...,11464,mindmatters.ai,Mind Matters,0.0,- Brain surgeon Dr. Michael Egnor and author D...,AI can never be more than a machine; the human...,[]
35,Feedly AI,Oxford Robotics Institute director discusses t...,https://news.google.com/rss/articles/CBMiogFBV...,NaN,NaN,118,True,success,https://www.therobotreport.com/oxford-robotics...,download/html/Oxford_Robotics_Institute_direct...,2025-09-28T12:30:55Z,download/text/Oxford_Robotics_Institute_direct...,8782,therobotreport.com,The Robot Report,0.0,"- Nick Hawes, director of the Oxford Robotics ...","Nick Hawes, a professor of robotics and AI at ...",[]
39,Feedly AI,OpenAI shows off Stargate data center in Texas,https://news.google.com/rss/articles/CBMioAFBV...,NaN,NaN,169,True,success,https://www.nwaonline.com/news/2025/sep/28/ope...,download/html/OpenAI_shows_off_Stargate_data_c...,2025-09-28T06:50:00Z,download/text/OpenAI_shows_off_Stargate_data_c...,8099,nwaonline.com,NWA Online,0.0,- OpenAI unveiled its massive Stargate AI data...,"ABILENE, Texas -- The afternoon sun was so hot...","[business, national, imported]"
42,Feedly AI,What corporate earnings calls reveal about the...,https://news.google.com/rss/articles/CBMikwFBV...,NaN,NaN,203,True,success,https://cepr.org/voxeu/columns/what-corporate-...,download/html/What_corporate_earnings_calls_re...,2025-09-28T21:09:09Z,download/text/What_corporate_earnings_calls_re...,7721,cepr.org,Center for Economic and Policy Research,0.0,"- Analysis of over 22,000 S&P 500 earnings cal...",The launch of ChatGPT in late 2022 marked a tu...,[]
44,Reddit,NVIDIA Just Solved The Hardest Problem in Phys...,https://youtu.be/7NF3CdXkm68,NaN,NaN,362,True,success,https://www.youtube.com/watch?v=7NF3CdXkm68,download/html/NVIDIA_Just_Solved_The_Hardest_P...,2025-09-28T21:38:13Z,download/text/NVIDIA_Just_Solved_The_Hardest_P...,7539,youtube.com,YouTube,0.0,- NVIDIA has reportedly solved a major challen...,❤️ Check out Lambda here and sign up for their...,[ai]
45,NewsAPI,ai-ebash 0.5.21,https://pypi.org/project/ai-ebash/0.5.21/,2025-09-27T17:05:04Z,NaN,553,True,success,https://pypi.org/project/ai-ebash/,download/html/ai-ebash_0.5.21.html,2025-09-28T21:34:06Z,download/text/ai-ebash_0.5.21.txt,7433,pypi.org,PyPI,0.0,- AI-eBash is a console utility that integrate...,Console utility for integrating artificial int...,[]
119,Reddit,Morgan Stanley warns AI could sink 42-year-old...,https://finance.yahoo.com/news/morgan-stanley-...,NaN,NaN,329,True,success,https://www.thestreet.com/technology/morgan-st...,download/html/Morgan_Stanley_warns_AI_could_si...,2025-09-25T18:03:00Z,download/text/Morgan_Stanley_warns_AI_could_si...,4290,thestreet.com,The Street,0.0,"- Morgan Stanley downgraded Adobe, citing skep...",Market braces for fallout if AI risk plays out,"[Creative Cloud, ARR, Generative AI, Morgan St..."
135,HackerNoon,AI Startup Turns Open Source Code Reviews Into...,https://hackernoon.com/ai-startup-turns-open-s...,"Sat, 27 Sep 2025 23:59:59 GMT",Awesome Reviewers turns real code review comme...,247,True,success,https:/

In [22]:
# User prompt to run workflow
# user_prompt = "Run step 5, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


16:37:14 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
16:37:14 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Using model 'gpt-4.1-mini' for topic extraction
16:37:14 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Processing 313 articles for topic extraction


▶ Starting Step 5: step_05_cluster_by_topic


16:37:23 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Successfully extracted 1183 total topics across articles
16:37:23 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Starting canonical topic classification for 101 topics
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/ca

INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized Langfu

INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_top

...

16:40:19 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | 19: AI Security Threats and Vulnerabilities
16:40:19 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Prompt injection – and a $5 domain – trick Salesforce Agentforce into leaking sales (Agentforce AI, Salesforce, AI Security, Vulnerabilities, Cybersecurity, Privacy, Governance)
Experts Alarmed That AI Is Now Producing Functional Viruses (AI-designed DNA, Bioweapons, Safety And Alignment, Policy And Regulation, AI Medicine, FDA Regulation, Pathogen Datasets)
OpenAI plugs ShadowLeak bug in ChatGPT that let miscreants raid inboxes (Vulnerabilities, AI Security, OpenAI, Privacy, Cybersecurity, Agents, Safety And Alignment)
Stop runaway AI before it's too late, experts beg the UN (AI Regulation, Safety And Alignment, Autonomous Weapons, UN Policies, Expert Advocacy, Disinformation, Governance)
ChatGPT joins human league, now solves CAPTCHAs for the right prompt (CAPTCHA Bypass, Prompt Injection, AI Secu

16:40:22 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | 21: AI Chatbots and Child Safety Regulation
16:40:22 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Tech companies under pressure as California governor weighs AI bills (AI Safety Legislation, Chatbot Regulation, Child Protection, Tech Lobbying, Policy And Regulation, Safety And Alignment, Mental Health)
'After colonizing the adult world, AI enters the world of children' (AI Toys, Toy Industry, Children's Products, Conversational AI, Language Models, Emotional Impact, Ethics)
After child’s trauma, chatbot maker allegedly forced mom to arbitration for $100 payout (Child Safety, Chatbots, Safety And Alignment, Lawsuits, Regulation, Parental Concerns)
Indiana congresswoman helps introduce first bill addressing AI chatbot companions (FTC Legislation, Children's Online Safety, Chatbots, Education, Policy And Regulation, Privacy, AI Safety)
Senators weigh regulating AI chatbots to protect kidsParents to

16:40:27 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | 

16:40:27 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | 1: AI Advances in Robotics and Automation
16:40:27 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Google DeepMind’s new AI models can search the web to help robots complete tasks (Gemini Robotics, Google DeepMind, Robotics AI, Robot Task Automation, Computer Vision, Agents)
“No AI was used to design it”: Elon Musk shares stunning visual of Starship straight out of sci-fi (SpaceX Starship, 3D Printing, Spacecraft Engineering, Mars Colonization, Manufacturing, Elon Musk)
Google DeepMind unveils its first “thinking” robotics AI (Gemini Robotics, Robots, Embodied Reasoning, Vision-Language Models, Agentic Robots, Reinforcement Learning, Artificial General Intelligence)
DeepMind’s robotic ballet: An AI for coordinating manufacturing robots (Industrial Robots, Manufacturing, Automation Systems, Job Automation, Hardware)
Google De

16:40:34 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | 

16:40:35 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | 0: China-US Tensions Over Semiconductor Chips
16:40:35 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Nvidia GeForced out of China as Beijing demands tech titans embrace homegrown silicon (GPU Restrictions, Export Controls, China, Nvidia, AI Hardware, Policy And Regulation, Semiconductor Chips)
China blocks sale of Nvidia AI chips (Semiconductor Ban, China, NVIDIA, AI Chips, China Tech, Policy And Regulation, Geopolitics)
China is ‘nanoseconds behind’ US in chips, says Nvidia’s Jensen Huang (Nvidia, Semiconductor Chips, US-China relations, Semiconductors: Geopolitics, US-China tech war, Semiconductors: Policy)
16:40:35 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | 



⏱️  Total execution time: 200.78s
📊 Final result:
✅ Step 5 step_05_cluster_by_topic completed successfully! Organized 313 articles into topic clusters.


In [7]:
# User prompt to run workflow
# user_prompt = "Run step 6, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

17:12:46 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Rating 313 AI articles using fn_rate_articles
17:12:46 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Calculating article ratings for 313 articles
17:12:46 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Rating recency
17:12:46 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Rating spam probability
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 6: step_06_rate_articles


17:12:53 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | low quality articles: {0.0: 253, 1.0: 26, 0.9241417710874382: 1, 0.999996587894339: 1, 0.9999723899264568: 1, 7.88926463665814e-06: 1, 0.9999920581810099: 1, 1.2098659934854867e-06: 1, 9.422445831379603e-07: 1, 1.5428112031918877e-13: 1, 0.7549149672612132: 1, 0.9859363206911979: 1, 3.6534822137210456e-08: 1, 1.8925145442106525e-05: 1, 7.433680672352188e-12: 1, 0.1329642264019799: 1, 0.9999989719621736: 1, 1.1399918530443554e-12: 1, 0.0001584362410990077: 1, 1.370956471489639e-06: 1, 2.3659776091347615e-14: 1, 3.2241867372567335e-08: 1, 3.737889382610936e-12: 1, 2.389552736575778e-13: 1, 2.8453348089834e-08: 1, 0.00070967033651176: 1, 0.047425874568977164: 1, 0.9399132588278407: 1, 1.162823303022097e-10: 1, 0.20181323122263276: 1, 6.34880011604368e-09: 1, 3.1608814543136926e-10: 1, 0.18242550282051248: 1, 0.9999998063873693: 1}
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/r

✅ Completed Step 6: Rated 311 articles
⏱️  Total execution time: 18.86s
📊 Final result:
✅ Step 6 step_06_rate_articles completed successfully! Rated 311 articles with average rating 4.2/10.
⭐ High quality articles (≥7.0): 14
💾 Ratings stored in persistent state.


In [8]:
state.headline_df

,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,...,cluster_name,input_text,age,recency_score,low_quality,on_topic,important,bt_z,adjusted_len,rating
0,New York Times,The Great A.I. Build-Out + H-1B Visa Chaos + T...,https://www.nytimes.com/2025/09/26/podcasts/ha...,"Fri, 26 Sep 2025 11:00:04 +0000",“You now have the leaders of the biggest techn...,0,True,success,https://www.nytimes.com/2025/09/26/podcasts/ha...,download/html/The_Great_A.I._Build-Out___H-1B_...,...,Other,The Great A.I. Build-Out + H-1B Visa Chaos + T...,1.000004,-0.000003,0.000000,1.000000e+00,1.0,0.0,1.549347,8.549344e+00
1,Bloomberg,AI Cloud Firm Northern Data Raided by German I...,https://www.bloomberg.com/news/articles/2025-0...,None,None,1,True,success,https://www.bloomberg.com/news/articles/2025-0...,download/html/AI_Cloud_Firm_Northern_Data_Raid...,...,Other,AI Cloud Firm Northern Data Raided by German I...,1.000004,-0.000003,0.000000,1.000000e+00,1.0,0.0,0.670246,7.670243e+00
2,FT,Accenture to ‘exit’ staff who cannot be retrai...,https://www.ft.com/content/a74f8564-ed5a-42e9-...,None,None,2,True,success,https://www.ft.com/content/a74f8564-ed5a-42e9-...,download/html/Accenture_to__exit__staff_who_ca...,...,AI's Impact on Jobs and Work,Accenture to ‘exit’ staff who cannot be retrai...,1.000004,-0.000003,0.000000,1.000000e+00,1.0,0.0,0.504335,7.504332e+00
3,New York Times,Nvidia to Invest $100 Billion in OpenAI,https://www.nytimes.com/2025/09/22/technology/...,"Mon, 22 Sep 2025 18:26:04 +0000",The chipmaker’s investment in the San Francisc...,3,True,success,https://www.nytimes.com/2025/09/22/technology/...,download/html/Nvidia_to_Invest__100_Billion_in...,...,Nvidia and OpenAI's $100B AI Investment,Nvidia to Invest $100 Billion in OpenAI\n- Nvi...,1.000004,-0.000003,0.000000,1.000000e+00,1.0,0.0,0.437592,7.437589e+00
4,New York Times,OpenAI to Join Tech Giants in Building 5 New D...,https://www.nytimes.com/2025/09/23/technology/...,"Tue, 23 Sep 2025 21:00:07 +0000",Working with the Japanese conglomerate SoftBan...,4,True,success,https://www.nytimes.com/2025/09/23/technology/...,download/html/OpenAI_to_Join_Tech_Giants_in_Bu...,...,OpenAI's Expansion of AI Data Centers,OpenAI to Join Tech Giants in Building 5 New D...,1.000004,-0.000003,0.000000,1.000000e+00,1.0,0.0,0.215638,7.215635e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,Feedly AI,OpenAI’s New Move: ChatGPT Knows What You Need...,https://www.pymnts.com/artificial-intelligence...,None,None,306,True,success,https://www.pymnts.com/artificial-intelligence...,download/html/OpenAI_s_New_Move__ChatGPT_Knows...,...,Other,OpenAI’s New Move: ChatGPT Knows What You Need...,1.926555,-0.473887,1.000000,0.000000e+00,0.0,0.0,0.608954,1.350670e-01
307,Feedly AI,SK Telecom Offers Deep Value And AI Optionalit...,https://seekingalpha.com/article/4826412-sk-te...,None,None,307,True,success,https://seekingalpha.com/article/4826412-sk-te...,download/html/SK_Telecom_Offers_Deep_Value_And...,...,AI Innovation and Investment Trends,SK Telecom Offers Deep Value And AI Optionalit...,0.000000,1.000000,1.000000,9.610239e-05,0.0,0.0,0.000000,9.610239e-05
308,Feedly AI,Box's Explosive AI Potential (NYSE:BOX),https://seekingalpha.com/article/4826458-boxs-...,None,None,308,True,success,https://seekingalpha.com/article/4826458-boxs-...,download/html/Box_s_Explosive_AI_Potential__NY...,...,AI Innovation and Investment Trends,Box's Explosive AI Potential (NYSE:BOX)\n- no ...,0.000000,1.000000,1.000000,4.058652e-10,0.0,0.0,0.000000,4.058652e-10
309,NewsAPI,Wall Street billionaire sends one-word AI warning,https://biztoc.com/x/15f00a2900063c5f,2025-09-27T18:17:17Z,None,309,True,success,https://biztoc.com/x/15f00a2900063c5f,download/html/Wall_Street_billionaire_sends_on...,...,Other,Wall Street billionaire sends one-word AI warn...,1.131786,-0.087299,0.999999,9.999998e-01,0.0,0.0,0.058426,-2.887240e-02


In [24]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


16:41:13 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Starting check_workflow_status
16:41:13 | NewsletterAgent.test_newsletter_20250928144400108448 | INFO | Completed check_workflow_status


⏱️  Total execution time: 5.07s
📊 Final result:
Current workflow status:
- Progress: 55.6% (5/9 complete)
- 5 complete, 0 started, 0 failed, 4 not started
- Next step: Step 6 — Rate Articles

Step details:
- Step 1: Fetch Urls — complete
- Step 2: Filter Urls — complete
- Step 3: Download Articles — complete
- Step 4: Extract Summaries — complete
- Step 5: Cluster By Topic — complete
- Step 6: Rate Articles — not_started
- Step 7: Select Sections — not_started
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

Data summary:
- Total articles: 313
- AI-related: 313
- Clusters: 0
- Sections: 0

Would you like me to run Step 6 (rate articles) now?


In [57]:
class SiteNameGeneration(BaseModel):
    """Single domain to site name mapping result"""
    id: int = Field(description="The site id")
    domain: str = Field(description="The domain name")
    site_name: str = Field(description="Canonical site name for the domain")


class SiteNameGenerationList(BaseModel):
    """List of SiteNameGeneration for batch processing"""
    results_list: list[SiteNameGeneration] = Field(
        description="List of site name generation results")


In [82]:

class StoryRating(BaseModel):
    """StoryRating class for generic structured output rating"""
    id: int = Field(description="The id of the story")
    rating: int = Field(description="An integer rating of the story")


class StoryRatings(BaseModel):
    """StoryRatings class for structured output filtering of a list of Story"""
    items: List[StoryRating] = Field(description="List of StoryRating")


In [136]:
class StoryOrder(BaseModel):
    """StoryOrder class for generic structured output rating"""
    id: int = Field(description="The id of the story")


class StoryOrderList(BaseModel):
    """List of StoryOrder for structured output"""
    items: List[StoryOrder] = Field(
        description="List of StoryOrder")

# Setup battle agent
system, user, model = LangfuseClient().get_prompt("newsagent/battle_prompt")
battle_agent = LLMagent(
    system_prompt=system,
    user_prompt=user,
    model=model,
    output_type=StoryOrder,
    verbose=False,
    logger=logger
)

    

INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/battle_prompt' from Langfuse
INFO:llm:Parsed prompt 'newsagent/battle_prompt': model=gpt-5-mini, system_len=2439, user_len=128


In [8]:
headline_df = state.headline_df

In [9]:
headline_df


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,...,content_length,domain,site_name,reputation,summary,description,topics,extended_summary,cluster,cluster_name
0,Feedly AI,The Role ofArtificial Intelligenceand Machine ...,https://www.cureus.com/articles/408345-the-rol...,None,None,0,True,success,https://www.cureus.com/articles/408345-the-rol...,download/html/The_Role_ofArtificial_Intelligen...,...,40476,cureus.com,Cureus,0.0,- AI and machine learning (ML) significantly i...,Cerebral small vessel disease (CSVD) contribut...,"[Cerebral Small Vessel Disease, Neuroimaging, ...",The Role ofArtificial Intelligenceand Machine ...,9,AI Applications in Science and Medicine
1,Feedly AI,iRobot co-founder Rodney Brooks details why hu...,https://rodneybrooks.com/why-todays-humanoids-...,None,None,1,True,success,https://rodneybrooks.com/why-todays-humanoids-...,download/html/iRobot_co-founder_Rodney_Brooks_...,...,39852,rodneybrooks.com,Rodney Brooks,0.0,- Despite significant funding and efforts by c...,,"[Humanoid Robots, Dexterity Challenges, Tactil...",iRobot co-founder Rodney Brooks details why hu...,-1,Other
2,New York Times,The Great A.I. Build-Out + H-1B Visa Chaos + T...,https://www.nytimes.com/2025/09/26/podcasts/ha...,"Fri, 26 Sep 2025 11:00:04 +0000",“You now have the leaders of the biggest techn...,2,True,success,https://www.nytimes.com/2025/09/26/podcasts/ha...,download/html/The_Great_A.I._Build-Out___H-1B_...,...,35428,nytimes.com,The New York Times,5.0,- NVIDIA and OpenAI announced a $100 billion i...,“You now have the leaders of the biggest techn...,"[AI Data Centers, Infrastructure, NVIDIA Corpo...",The Great A.I. Build-Out + H-1B Visa Chaos + T...,-1,Other
3,HackerNoon,Knowledge Graphs Gain Traction as AI Pushes Be...,https://hackernoon.com/knowledge-graphs-gain-t...,"Thu, 25 Sep 2025 05:05:23 GMT",Is graph really the new star schema? What do g...,3,True,success,https://hackernoon.com/knowledge-graphs-gain-t...,download/html/Knowledge_Graphs_Gain_Traction_a...,...,35258,hackernoon.com,Hacker Noon,2.0,- Knowledge Graphs are increasingly recognized...,Is graph really the new star schema? What do g...,"[Knowledge Graphs, Enterprise AI, Ontology Eng...",Knowledge Graphs Gain Traction as AI Pushes Be...,-1,Other
4,HackerNoon,Building GPT-2 from Scratch in Rust - A Softwa...,https://hackernoon.com/building-gpt-2-from-scr...,"Thu, 25 Sep 2025 04:23:34 GMT",Rust is a Rust-based programming language. It'...,4,True,success,https://hackernoon.com/building-gpt-2-from-scr...,download/html/Building_GPT-2_from_Scratch_in_R...,...,18936,hackernoon.com,Hacker Noon,2.0,- A software engineer built a working GPT-2 mo...,Learn how a software engineer built a working ...,"[GPT-2 Implementation, Rust Programming, Trans...",Building GPT-2 from Scratch in Rust - A Softwa...,-1,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,WSJ,A New Front Opens Between Zuckerberg and Musk ...,https://www.wsj.com/tech/ai/meta-tesla-robots-...,None,None,308,True,success,https://www.wsj.com/tech/ai/meta-tesla-robots-...,,...,0,wsj.com,The Wall Street Journal,5.0,- no content,,[],A New Front Opens Between Zuckerberg and Musk ...,-1,Other
309,WSJ,Walmart CEO Issues Wake-Up Call: ‘AI Is Going ...,https://www.wsj.com/tech/ai/walmart-ceo-doug-m...,None,None,309,True,success,https://www.wsj.com/tech/ai/walmart-ceo-doug-m...,,...,0,wsj.com,The Wall Street Journal,5.0,- no content,,[],Walmart CEO Issues Wake-Up Call: ‘AI Is Going ...,-1,Other
310,WSJ,What Are ‘World Models’? The Key to the Next B...,https://www.wsj.com/tech/ai/world-models-ai-ev...,None,None,310,True,success,https://www.wsj.com/tech/ai/world-models-ai-ev...,,...,0,wsj.com,The Wall Street Journal,5.0,- no content,,[],What Are ‘World Models’? The Key to the Next B...,4,AI Innovation and Investment Trends
311,WSJ,"For Mistral, the Future of AI Development Will...",https://www.wsj.com/articles/for-mistral-the-f...,None,None,311,True,su

In [13]:
    headline_df['adjusted_len'] = headline_df['content_length'].clip(lower=1)


In [14]:
from do_rating import *
zdf = await fn_rate_articles(headline_df, logger)


17:06:03 | NewsletterAgent.newsletter_agent | INFO | Calculating article ratings for 313 articles
17:06:03 | NewsletterAgent.newsletter_agent | INFO | Rating recency
17:06:03 | NewsletterAgent.newsletter_agent | INFO | Rating spam probability
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246
17:06:12 | NewsletterAgent.newsletter_agent | INFO | low quality articles: {0.0: 254, 1.0: 25, 0.9947799085613173: 2, 0.9999998063873693: 2, 0.49999999904767284: 2, 7.991959892953932e-11: 1, 2.020683909022217e-11: 1, 0.9875682652714307: 1, 1.1253517471925912e-07: 1, 0.7549148997665586: 1, 4.8473687062702556e-11: 1, 0.9999038520641101: 1, 0.9149009412616036: 1, 0.000139822025552885: 1, 0.022977367413042835: 1, 0.09534945969074979: 1, 3.2661313427874473e-13: 1, 1.006039279283317e-12: 1, 0.24508502137530588: 1, 1.2256067444099483e-11: 1, 9.610239

In [16]:
zdf 


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,...,cluster_name,adjusted_len,input_text,age,recency_score,low_quality,on_topic,important,bt_z,rating
0,New York Times,The Great A.I. Build-Out + H-1B Visa Chaos + T...,https://www.nytimes.com/2025/09/26/podcasts/ha...,"Fri, 26 Sep 2025 11:00:04 +0000",“You now have the leaders of the biggest techn...,0,True,success,https://www.nytimes.com/2025/09/26/podcasts/ha...,download/html/The_Great_A.I._Build-Out___H-1B_...,...,Other,1.549347,The Great A.I. Build-Out + H-1B Visa Chaos + T...,1.000004,-0.000003,0.000000,1.000000e+00,1.0,0.0,8.549344e+00
1,Bloomberg,AI Cloud Firm Northern Data Raided by German I...,https://www.bloomberg.com/news/articles/2025-0...,None,None,1,True,success,https://www.bloomberg.com/news/articles/2025-0...,download/html/AI_Cloud_Firm_Northern_Data_Raid...,...,Other,0.670246,AI Cloud Firm Northern Data Raided by German I...,1.000004,-0.000003,0.000000,1.000000e+00,1.0,0.0,7.670243e+00
2,FT,Accenture to ‘exit’ staff who cannot be retrai...,https://www.ft.com/content/a74f8564-ed5a-42e9-...,None,None,2,True,success,https://www.ft.com/content/a74f8564-ed5a-42e9-...,download/html/Accenture_to__exit__staff_who_ca...,...,AI's Impact on Jobs and Work,0.504335,Accenture to ‘exit’ staff who cannot be retrai...,1.000004,-0.000003,0.000000,1.000000e+00,1.0,0.0,7.504332e+00
3,New York Times,Nvidia to Invest $100 Billion in OpenAI,https://www.nytimes.com/2025/09/22/technology/...,"Mon, 22 Sep 2025 18:26:04 +0000",The chipmaker’s investment in the San Francisc...,3,True,success,https://www.nytimes.com/2025/09/22/technology/...,download/html/Nvidia_to_Invest__100_Billion_in...,...,Nvidia and OpenAI's $100B AI Investment,0.437592,Nvidia to Invest $100 Billion in OpenAI\n- Nvi...,1.000004,-0.000003,0.000000,1.000000e+00,1.0,0.0,7.437589e+00
4,New York Times,OpenAI to Join Tech Giants in Building 5 New D...,https://www.nytimes.com/2025/09/23/technology/...,"Tue, 23 Sep 2025 21:00:07 +0000",Working with the Japanese conglomerate SoftBan...,4,True,success,https://www.nytimes.com/2025/09/23/technology/...,download/html/OpenAI_to_Join_Tech_Giants_in_Bu...,...,OpenAI's Expansion of AI Data Centers,0.215638,OpenAI to Join Tech Giants in Building 5 New D...,1.000004,-0.000003,0.000000,1.000000e+00,1.0,0.0,7.215635e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,Feedly AI,OpenAI’s New Move: ChatGPT Knows What You Need...,https://www.pymnts.com/artificial-intelligence...,None,None,306,True,success,https://www.pymnts.com/artificial-intelligence...,download/html/OpenAI_s_New_Move__ChatGPT_Knows...,...,Other,0.608954,OpenAI’s New Move: ChatGPT Knows What You Need...,1.921890,-0.472183,1.000000,0.000000e+00,0.0,0.0,1.367707e-01
307,Feedly AI,SK Telecom Offers Deep Value And AI Optionalit...,https://seekingalpha.com/article/4826412-sk-te...,None,None,307,True,success,https://seekingalpha.com/article/4826412-sk-te...,download/html/SK_Telecom_Offers_Deep_Value_And...,...,AI Innovation and Investment Trends,0.000000,SK Telecom Offers Deep Value And AI Optionalit...,0.000000,1.000000,1.000000,2.144494e-05,0.0,0.0,2.144494e-05
308,Feedly AI,Box's Explosive AI Potential (NYSE:BOX),https://seekingalpha.com/article/4826458-boxs-...,None,None,308,True,success,https://seekingalpha.com/article/4826458-boxs-...,download/html/Box_s_Explosive_AI_Potential__NY...,...,AI Innovation and Investment Trends,0.000000,Box's Explosive AI Potential (NYSE:BOX)\n- no ...,0.000000,1.000000,1.000000,1.006039e-12,0.0,0.0,1.006084e-12
309,NewsAPI,Wall Street billionaire sends one-word AI warning,https://biztoc.com/x/15f00a2900063c5f,2025-09-27T18:17:17Z,None,309,True,success,https://biztoc.com/x/15f00a2900063c5f,download/html/Wall_Street_billionaire_sends_on...,...,Other,0.058426,Wall Street billionaire sends one-word AI warn...,1.127122,-0.084344,0.999904,9.999977e-01,0.0,0.0,-2.582384e-02


In [27]:
    rating_df = headline_df.copy().fillna({
        'content_length': 1,
        'reputation': 0,
        'on_topic': 0,
        'importance': 0,
        'low_quality': 0,
    })

In [28]:
    rating_df['title'] = rating_df['title'].fillna("")
    rating_df['title'] = rating_df['title'].astype(str)
    rating_df['summary'] = rating_df['summary'].astype(str)
    rating_df['summary'] = rating_df['summary'].fillna("")


In [29]:
    rating_df['input_text'] = rating_df['title'] + "\n" + rating_df['summary']


In [31]:
from datetime import datetime, timezone, timedelta

yesterday = (datetime.now(timezone.utc)
                 - timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%SZ")
rating_df['last_updated'] = rating_df['last_updated'].fillna(yesterday)
rating_df["age"] = (datetime.now(timezone.utc) -
                    pd.to_datetime(rating_df['last_updated']))
rating_df["age"] = rating_df["age"].dt.total_seconds() / (24 * 60 * 60)
rating_df["age"] = rating_df["age"].clip(lower=0)  # no negative dates
# only consider articles from the last week
rating_df = rating_df[rating_df["age"] < 7].copy()
k = np.log(2)  # 1/2 after 1 day
# 1 point at age 0, 0 at age 1, -0.5 at age 2, -1 at age infinity
rating_df["recency_score"] = 2 * np.exp(-k * rating_df["age"]) - 1


In [33]:

class StoryRating(BaseModel):
    """StoryRating class for generic structured output rating"""
    id: int = Field(description="The id of the story")
    rating: int = Field(description="An integer rating of the story")


class StoryRatings(BaseModel):
    """StoryRatings class for structured output filtering of a list of Story"""
    items: List[StoryRating] = Field(description="List of StoryRating")


In [34]:
        system, user, model = LangfuseClient().get_prompt("newsagent/rate_quality")

        quality_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=StoryRatings,
            model=model,
            verbose=False,
            logger=logger
        )

        rating_df['low_quality'] = await quality_agent.filter_dataframe(
            rating_df[['id', 'input_text']],
            value_field='low_quality',
            item_list_field='results_list',
            item_id_field='id',
            chunk_size=25,
            return_probabilities=True
        )

INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


In [35]:
    counts = rating_df["low_quality"].value_counts().to_dict()
    if logger:
        logger.info(f"low quality articles: {counts}")


16:49:56 | NewsletterAgent.newsletter_agent | INFO | low quality articles: {0.0: 252, 1.0: 26, 2.699578503363014e-07: 2, 0.9999930118027327: 1, 0.9740425389805024: 1, 2.215948977336598e-08: 1, 0.9999545100305701: 1, 0.0001233946226544279: 1, 0.008577484930474293: 1, 1.2790354113010132e-13: 1, 0.9998765647550606: 1, 0.9959299041310293: 1, 0.047425874568977164: 1, 2.707717986972444e-13: 1, 6.914400106940203e-13: 1, 0.3208212708736474: 1, 0.9999987335551229: 1, 5.2792096283383845e-15: 1, 1.147876852587815e-05: 1, 3.4767787164951853e-13: 1, 4.5990553786523166e-10: 1, 0.999998137537802: 1, 4.8473687062702556e-11: 1, 1.3615261080896538e-13: 1, 3.737889382610936e-12: 1, 5.675685232632723e-14: 1, 0.0003353500342255387: 1, 3.927863545481039e-07: 1, 0.377540667992942: 1, 0.867035682761771: 1, 1.0129987330277146e-05: 1, 0.017986211314027313: 1, 0.0019267342527389902: 1, 0.9706877045032167: 1}


In [38]:
        system, user, model = LangfuseClient().get_prompt("newsagent/rate_on_topic")

        topic_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=StoryRatings,
            model=model,
            verbose=False,
            logger=logger
        )

        rating_df['on_topic'] = await topic_agent.filter_dataframe(
            rating_df[['id', 'input_text']],
            value_field='on_topic',
            item_list_field='results_list',
            item_id_field='id',
            chunk_size=25,
            return_probabilities=True
        )

INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240


In [39]:
    counts = rating_df["low_quality"].value_counts().to_dict()
    if logger:
        logger.info(f"low quality articles: {counts}")


16:53:46 | NewsletterAgent.newsletter_agent | INFO | low quality articles: {0.0: 252, 1.0: 26, 2.699578503363014e-07: 2, 0.9999930118027327: 1, 0.9740425389805024: 1, 2.215948977336598e-08: 1, 0.9999545100305701: 1, 0.0001233946226544279: 1, 0.008577484930474293: 1, 1.2790354113010132e-13: 1, 0.9998765647550606: 1, 0.9959299041310293: 1, 0.047425874568977164: 1, 2.707717986972444e-13: 1, 6.914400106940203e-13: 1, 0.3208212708736474: 1, 0.9999987335551229: 1, 5.2792096283383845e-15: 1, 1.147876852587815e-05: 1, 3.4767787164951853e-13: 1, 4.5990553786523166e-10: 1, 0.999998137537802: 1, 4.8473687062702556e-11: 1, 1.3615261080896538e-13: 1, 3.737889382610936e-12: 1, 5.675685232632723e-14: 1, 0.0003353500342255387: 1, 3.927863545481039e-07: 1, 0.377540667992942: 1, 0.867035682761771: 1, 1.0129987330277146e-05: 1, 0.017986211314027313: 1, 0.0019267342527389902: 1, 0.9706877045032167: 1}


- rate articles
- load sources into db with reputation
- get domain from each url and put in headline_df
- look up source and reputation
- prompt for on topic , important, high quality
- run bradley terry
- combine ratings


set selected flag using rating
store to db

- next steps select sections
- clean clusters , combine clusters, select sections 

In [ ]:
import sys
import dotenv
dotenv.load_dotenv()

# Delete the module from cache
if 'llm' in sys.modules:
    print("llm")
    del sys.modules['llm']

# Now you can import it again
import llm
from llm import LLMagent


In [ ]:
agent = LLMagent(
  system_prompt="Classify as funny or not funny. Return only 1 for funny, 0 for not funny",
  user_prompt="Text: {text}\n",
  output_type=str,  # Not used for logprobs
  model="gpt-4.1-mini"
)

In [ ]:
_, logprobs = await agent.prompt_dict_chat_probs({'text': 'take my wife. please.'})
logprobs 



In [ ]:
logprobs.content[0].logprob



In [ ]:
if not logprobs or getattr(logprobs, 'content', None) is None:
            raise ValueError("Invalid logprobs_data. Must contain 'content' key with non-None value.")


In [ ]:
        first_token_logprobs = logprobs.content[0]


In [ ]:
first_token_logprobs.top_logprobs


In [ ]:
top_logprobs = first_token_logprobs.top_logprobs
top_logprobs

In [ ]:
agent._extract_token_probabilities(logprobs, "1") 


In [ ]:
await agent.run_prompt_with_probs(text="fruit flies like a banana")

In [ ]:
import pandas as pd

df = pd.DataFrame({
      "text": [
          "fruit flies like a banana",
          "yo momma so low she plays squash against the curb",
          "thou shalt not kill",
          "first came the thunder, then came the rain"
      ]
  })

In [ ]:
df 


In [ ]:
await agent.filter_dataframe(
      df[["text"]],
      return_probabilities=True,
      target_tokens=["1"]
  )


In [7]:
state.headline_df

,id,source,title,url,published,rss_summary,isAI,status,final_url,html_path,last_updated,text_path,content_length,summary,description,topics
0,191,Ars Technica,ChatGPT Pulse delivers morning updates based o...,https://arstechnica.com/ai/2025/09/chatgpt-pul...,"Thu, 25 Sep 2025 20:30:52 +0000",New mobile chatbot feature analyzes conversati...,True,success,https://arstechnica.com/ai/2025/09/chatgpt-pul...,download/html/ChatGPT_Pulse_delivers_morning_u...,2025-09-25T20:30:52Z,download/text/ChatGPT_Pulse_delivers_morning_u...,2521,"- OpenAI launched ChatGPT Pulse, a new mobile ...",New mobile chatbot feature analyzes conversati...,"[ChatGPT Pulse, Personalization, Asynchronous ..."
1,182,Ars Technica,Experts urge caution about using ChatGPT to pi...,https://arstechnica.com/information-technology...,"Thu, 25 Sep 2025 18:10:50 +0000",AI-selected portfolios might perform well in a...,True,success,https://arstechnica.com/information-technology...,download/html/Experts_urge_caution_about_using...,2025-09-25T18:10:50Z,download/text/Experts_urge_caution_about_using...,2611,- At least 13% of retail investors use AI tool...,AI-selected portfolios might perform well in a...,"[Retail Investors, Investment Tools, AI in Fin..."
2,202,Ars Technica,Google DeepMind unveils its first “thinking” r...,https://arstechnica.com/google/2025/09/google-...,"Thu, 25 Sep 2025 16:00:59 +0000",DeepMind researchers believe this is the dawn ...,True,success,https://arstechnica.com/google/2025/09/google-...,download/html/Google_DeepMind_unveils_its_firs...,2025-09-25T16:00:59Z,download/text/Google_DeepMind_unveils_its_firs...,2435,- Google DeepMind introduced Gemini Robotics 1...,DeepMind researchers believe this is the dawn ...,"[DeepMind, Robotics, AI Models, Agents, Comput..."
3,206,Ars Technica,DeepMind’s robotic ballet: An AI for coordinat...,https://arstechnica.com/science/2025/09/deepmi...,"Thu, 25 Sep 2025 11:15:40 +0000",An AI figures out how robots can get jobs done...,True,success,https://arstechnica.com/science/2025/09/deepmi...,download/html/DeepMind_s_robotic_ballet__An_AI...,None,download/text/DeepMind_s_robotic_ballet__An_AI...,2346,"- DeepMind developed RoboBallet, an AI system ...",An AI figures out how robots can get jobs done...,"[DeepMind, Manufacturing Robots, AI Coordinati..."
4,199,Ars Technica,Why does OpenAI need six giant data centers?,https://arstechnica.com/ai/2025/09/why-does-op...,"Wed, 24 Sep 2025 16:06:03 +0000",OpenAI's new $400 billion announcement reveals...,True,success,https://arstechnica.com/ai/2025/09/why-does-op...,download/html/Why_does_OpenAI_need_six_giant_d...,None,download/text/Why_does_OpenAI_need_six_giant_d...,2461,"- OpenAI, Oracle, and SoftBank are developing ...",OpenAI’s new $400 billion announcement reveals...,"[Stargate Project, AI Data Centers, AI Infrast..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,229,NewsAPI,From 4% To 20%: Women's Enrolment Goes Up Four...,https://www.ndtvprofit.com/technology/from-4-t...,2025-09-25T13:19:16Z,None,True,success,https://www.ndtvprofit.com/technology/from-4-t...,download/html/From_4__To_20___Women_s_Enrolmen...,2025-09-25T13:19:16Z,download/text/From_4__To_20___Women_s_Enrolmen...,2004,- Women's enrolment in AI and machine learning...,Up to 70% of women candidates who enrolled in ...,"[Women in Tech, AI Education, India, STEM Enro..."
319,22,NewsAPI,I met Sam Altman in Texas. He’s turning the ra...,https://finance.yahoo.com/news/met-sam-altman-...,2025-09-25T16:29:49Z,None,True,success,https://finance.yahoo.com/news/met-sam-altman-...,download/html/I_met_Sam_Altman_in_Texas._He_s_...,2025-09-25T16:29:49Z,download/text/I_met_Sam_Altman_in_Texas._He_s_...,9182,"- OpenAI, led by Sam Altman, is undertaking th...","In Abilene, I saw how tech companies like Open...","[AI Infrastructure, Data Centers, Compute Capa..."
320,74,NewsAPI,Google’s Data Commons MCP Server Anchors AI in...,http://www.pymnts.com/news/artificial-intellig...,2025-09-25T16:00:29

In [8]:
from do_rating import *

In [9]:
rating_df = state.headline_df.copy().fillna({
        'article_len': 1,
        'reputation': 0,
        'on_topic': 0,
        'importance': 0,
        'low_quality': 0,
    })

In [10]:
    # Ensure 'title' and 'summary' are always strings
    rating_df['title'] = rating_df['title'].fillna("")
    rating_df['title'] = rating_df['title'].astype(str)
    rating_df['summary'] = rating_df['summary'].astype(str)
    rating_df['summary'] = rating_df['summary'].fillna("")


In [23]:
    rating_df['input_text'] = rating_df['title'] + "\n" + rating_df['summary']


In [12]:
    yesterday = (datetime.now(timezone.utc)
                 - timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%SZ")
    rating_df['last_updated'] = rating_df['last_updated'].fillna(yesterday)
    rating_df["age"] = (datetime.now(timezone.utc) -
                        pd.to_datetime(rating_df['last_updated']))
    rating_df["age"] = rating_df["age"].dt.total_seconds() / (24 * 60 * 60)
    rating_df["age"] = rating_df["age"].clip(lower=0)  # no negative dates
    # only consider articles from the last week
    rating_df = rating_df[rating_df["age"] < 7].copy()
    k = np.log(2)  # 1/2 after 1 day
    # 1 point at age 0, 0 at age 1, -0.5 at age 2, -1 at age infinity
    rating_df["recency_score"] = 2 * np.exp(-k * rating_df["age"]) - 1


In [16]:
print(rating_df.loc[9].input_str)


AI medical tools found to downplay symptoms of women, ethnic minorities
- Research from MIT and the London School of Economics finds that large language model-based AI medical tools frequently downplay symptoms in female, Black, and Asian patients, leading to biased and less empathetic care recommendations.
- These AI models, including OpenAI's GPT-4, Meta's Llama 3, and Google's Gemma, have been shown to suggest lower levels of care for women and less compassionate guidance for minority groups, potentially worsening existing disparities in healthcare outcomes.
- Despite AI's growing adoption by major tech companies and health systems to aid overburdened physicians and improve diagnostics, experts warn that such biases could reinforce under-treatment patterns unless addressed by future development and deployment practices.


In [17]:
        system, user, model = LangfuseClient().get_prompt("newsagent/rate_quality")


INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


In [18]:
class StoryRating(BaseModel):
    """StoryRating class for generic structured output rating"""
    id: int = Field(description="The id of the story")
    rating: int = Field(description="An integer rating of the story")


class StoryRatings(BaseModel):
    """StoryRatings class for structured output filtering of a list of Story"""
    items: List[StoryRating] = Field(description="List of StoryRating")


        

In [19]:

        quality_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=StoryRatings,
            model=model,
            verbose=False,
            logger=logger
        )
    

In [24]:

        rating_df['low_quality'] = await quality_agent.filter_dataframe(
            rating_df[['id', 'input_text']],
            value_field='low_quality',
            item_list_field='results_list',
            item_id_field='id',
            chunk_size=25,
            return_probabilities=True
        )

In [27]:
rating_df.loc[rating_df["low_quality"]>0].sort_values("low_quality")


,id,source,title,url,published,rss_summary,isAI,status,final_url,html_path,...,text_path,content_length,summary,description,topics,input_str,age,recency_score,input_text,low_quality
92,32,Feedly AI,BetterArtificial Intelligence(AI) Stock: Nebiu...,https://www.nasdaq.com/articles/better-artific...,None,None,True,success,https://www.nasdaq.com/articles/better-artific...,download/html/BetterArtificial_Intelligence_AI...,...,download/text/BetterArtificial_Intelligence_AI...,7829,- Nebius and CoreWeave have experienced extrao...,Key PointsThe demand for cloud AI infrastructu...,"[Cloud AI Infrastructure, CoreWeave, Microsoft...",BetterArtificial Intelligence(AI) Stock: Nebiu...,1.000008,-0.000005,BetterArtificial Intelligence(AI) Stock: Nebiu...,6.041735e-14
317,269,NewsAPI,"Love, lies, and algorithms: Is AI really helpi...",https://biztoc.com/x/aeeb14c7b5385560,2025-09-25T15:37:55Z,None,True,success,https://biztoc.com/x/aeeb14c7b5385560,download/html/Love__lies__and_algorithms__Is_A...,...,download/text/Love__lies__and_algorithms__Is_A...,1276,- AI is increasingly integrated into dating ap...,From dating apps and AI-powered matchmaking to...,"[Dating Apps, Matchmaking, Personal Relationsh...","Love, lies, and algorithms: Is AI really helpi...",1.471756,-0.278913,"Love, lies, and algorithms: Is AI really helpi...",8.258073e-14
195,91,The Register,Microsoft insists Copilot+ PCs are 'empowering...,https://go.theregister.com/feed/www.theregiste...,2025-09-19T16:02:07.00Z,<h4>Latest marketing blitz for a solution seek...,True,success,https://go.theregister.com/feed/www.theregiste...,download/html/Microsoft_insists_Copilot__PCs_a...,...,download/text/Microsoft_insists_Copilot__PCs_a...,4555,- Microsoft promotes Copilot+ PCs and Windows ...,Comment: Latest marketing blitz for a solution...,"[Copilot PCs, Windows on Arm, Microsoft, PC Ha...",Microsoft insists Copilot+ PCs are 'empowering...,1.000008,-0.000005,Microsoft insists Copilot+ PCs are 'empowering...,8.258073e-14
129,142,HackerNoon,"Thirty Reports, Zero News: The AI PR Machine H...",https://hackernoon.com/thirty-reports-zero-new...,"Fri, 26 Sep 2025 05:10:27 GMT",The obsession with AI as a headline in itself ...,True,success,https://hackernoon.com/thirty-reports-zero-new...,download/html/Thirty_Reports__Zero_News__The_A...,...,download/text/Thirty_Reports__Zero_News__The_A...,3458,- Over thirty AI reports from PR agencies were...,"AI studies, surveys, forecasts, and whitepaper...","[AI Journalism, PR Agencies, Media Coverage, D...","Thirty Reports, Zero News: The AI PR Machine H...",0.374174,0.543094,"Thirty Reports, Zero News: The AI PR Machine H...",1.361526e-13
145,80,HackerNoon,My First Python Web App—Built in a Weekend (Wi...,https://hackernoon.com/my-first-python-web-app...,"Tue, 23 Sep 2025 06:28:17 GMT",Having an AI explain patterns and answer quest...,True,success,https://hackernoon.com/my-first-python-web-app...,download/html/My_First_Python_Web_App_Built_in...,...,download/text/My_First_Python_Web_App_Built_in...,4886,- Developer with no prior Python web experienc...,Having an AI explain patterns and answer quest...,"[AI Assistance, Coding Assistants, FastAPI, OA...",My First Python Web App—Built in a Weekend (Wi...,1.000008,-0.000005,My First Python Web App—Built in a Weekend (Wi...,1.981009e-13
89,278,Feedly AI,Nvidia's $100B OpenAI investment fuels AI bubb...,https://www.axios.com/2025/09/25/nvidia-openai...,None,None,True,success,https://www.axios.com/2025/09/25/nvidia-openai...,download/html/Nvidia_s__100B_OpenAI_investment...,...,download/text/Nvidia_s__100B_OpenAI_investment...,1214,- Nvidia's $100 billion investment in OpenAI h...,Wall Street is already concerned about an AI b...,"[Nvidia, OpenAI, Funding, Market Bubble, AI In...",Nvidia's $100B OpenAI investment fuels AI bubb...,1.000008,-0.000005,Nvidia's $100B OpenAI investment fuels AI bubb...,1.388794e-11
105,65,Feedly AI,I tried this Google Gemini feature — it's a ch...,https://www.tomsguide.com/ai/google-

In [28]:
        system, user, model = LangfuseClient().get_prompt("newsagent/rate_on_topic")


INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1780, user_len=240


In [29]:

        topic_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=StoryRatings,
            model=model,
            verbose=False,
            logger=logger
        )

        rating_df['on_topic'] = await topic_agent.filter_dataframe(
            rating_df[['id', 'input_text']],
            value_field='on_topic',
            item_list_field='results_list',
            item_id_field='id',
            chunk_size=25,
            return_probabilities=True
        )
        

In [30]:
rating_df


,id,source,title,url,published,rss_summary,isAI,status,final_url,html_path,...,content_length,summary,description,topics,input_str,age,recency_score,input_text,low_quality,on_topic
0,191,Ars Technica,ChatGPT Pulse delivers morning updates based o...,https://arstechnica.com/ai/2025/09/chatgpt-pul...,"Thu, 25 Sep 2025 20:30:52 +0000",New mobile chatbot feature analyzes conversati...,True,success,https://arstechnica.com/ai/2025/09/chatgpt-pul...,download/html/ChatGPT_Pulse_delivers_morning_u...,...,2521,"- OpenAI launched ChatGPT Pulse, a new mobile ...",New mobile chatbot feature analyzes conversati...,"[ChatGPT Pulse, Personalization, Asynchronous ...",ChatGPT Pulse delivers morning updates based o...,1.262820,-0.166543,ChatGPT Pulse delivers morning updates based o...,0.0,1.000000e+00
1,182,Ars Technica,Experts urge caution about using ChatGPT to pi...,https://arstechnica.com/information-technology...,"Thu, 25 Sep 2025 18:10:50 +0000",AI-selected portfolios might perform well in a...,True,success,https://arstechnica.com/information-technology...,download/html/Experts_urge_caution_about_using...,...,2611,- At least 13% of retail investors use AI tool...,AI-selected portfolios might perform well in a...,"[Retail Investors, Investment Tools, AI in Fin...",Experts urge caution about using ChatGPT to pi...,1.360066,-0.220871,Experts urge caution about using ChatGPT to pi...,0.0,9.999687e-01
2,202,Ars Technica,Google DeepMind unveils its first “thinking” r...,https://arstechnica.com/google/2025/09/google-...,"Thu, 25 Sep 2025 16:00:59 +0000",DeepMind researchers believe this is the dawn ...,True,success,https://arstechnica.com/google/2025/09/google-...,download/html/Google_DeepMind_unveils_its_firs...,...,2435,- Google DeepMind introduced Gemini Robotics 1...,DeepMind researchers believe this is the dawn ...,"[DeepMind, Robotics, AI Models, Agents, Comput...",Google DeepMind unveils its first “thinking” r...,1.450239,-0.268079,Google DeepMind unveils its first “thinking” r...,0.0,1.000000e+00
3,206,Ars Technica,DeepMind’s robotic ballet: An AI for coordinat...,https://arstechnica.com/science/2025/09/deepmi...,"Thu, 25 Sep 2025 11:15:40 +0000",An AI figures out how robots can get jobs done...,True,success,https://arstechnica.com/science/2025/09/deepmi...,download/html/DeepMind_s_robotic_ballet__An_AI...,...,2346,"- DeepMind developed RoboBallet, an AI system ...",An AI figures out how robots can get jobs done...,"[DeepMind, Manufacturing Robots, AI Coordinati...",DeepMind’s robotic ballet: An AI for coordinat...,1.000008,-0.000005,DeepMind’s robotic ballet: An AI for coordinat...,0.0,1.000000e+00
4,199,Ars Technica,Why does OpenAI need six giant data centers?,https://arstechnica.com/ai/2025/09/why-does-op...,"Wed, 24 Sep 2025 16:06:03 +0000",OpenAI's new $400 billion announcement reveals...,True,success,https://arstechnica.com/ai/2025/09/why-does-op...,download/html/Why_does_OpenAI_need_six_giant_d...,...,2461,"- OpenAI, Oracle, and SoftBank are developing ...",OpenAI’s new $400 billion announcement reveals...,"[Stargate Project, AI Data Centers, AI Infrast...",Why does OpenAI need six giant data centers?\n...,1.000008,-0.000005,Why does OpenAI need six giant data centers?\n...,0.0,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,229,NewsAPI,From 4% To 20%: Women's Enrolment Goes Up Four...,https://www.ndtvprofit.com/technology/from-4-t...,2025-09-25T13:19:16Z,None,True,success,https://www.ndtvprofit.com/technology/from-4-t...,download/html/From_4__To_20___Women_s_Enrolmen...,...,2004,- Women's enrolment in AI and machine learning...,Up to 70% of women candidates who enrolled in ...,"[Women in Tech, AI Education, India, STEM Enro...",From 4% To 20%: Women's Enrolment Goes Up Four...,1.562543,-0.322892,From 4% To 20%: Women's Enrolment Goes Up Four...,0.0,9.610239e-05
319,22,NewsAPI,I met Sam Altman in Texas. He’s turning the ra...,https://finance.yahoo.com/news/met-sam-altman-..

In [31]:
        system, user, model = LangfuseClient().get_prompt("newsagent/rate_importance")


INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_importance' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_importance': model=gpt-4.1, system_len=2145, user_len=252


In [32]:

        importance_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=StoryRatings,
            model=model,
            verbose=False,
            logger=logger
        )
    

In [33]:

        rating_df['important'] = await importance_agent.filter_dataframe(
            rating_df[['id', 'input_text']],
            value_field='important',
            item_list_field='results_list',
            item_id_field='id',
            chunk_size=25,
            return_probabilities=True
        )
    

In [34]:
    rating_df['bt_z'] = 0.0


In [35]:
    rating_df['adjusted_len'] = np.log10(rating_df['content_length']) - 3
    rating_df['adjusted_len'] = rating_df['adjusted_len'].clip(
        lower=0, upper=2)


/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [38]:
    rating_df['rating'] = rating_df['adjusted_len'] \
        + rating_df['on_topic'] \
        + rating_df['important'] \
        - rating_df['low_quality'] \
        + rating_df['bt_z'] \
        + rating_df['recency_score']

In [ ]:
todo: reputation 
bradley_terry